In [1]:
import pandas as pd
import pyecharts.options as opts
from pyecharts.charts import Line
import akshare as ak

In [2]:
# 财新制造业PMI https://akshare.akfamily.xyz/data/index/index.html#id54
pmi_cx_df = ak.index_pmi_man_cx()

# 股票指数的历史数据 https://akshare.akfamily.xyz/data/index/index.html#id4
stock_zh_index_daily_df = ak.stock_zh_index_daily(symbol="sh000001") # 'sh000001' 上证指数
stock_zh_index_daily_df['date'] = pd.to_datetime(stock_zh_index_daily_df['date'])
# 日频数据重采样，转为月频数据
stock_zh_index_monthly_df = stock_zh_index_daily_df.resample('M', on='date').agg({
    'open':'first',
    'high':'max',
    'low':'min',
    'close':'last',
    'volume':'sum'
})
# print(stock_zh_index_monthly_df)

In [4]:
x_data = pd.to_datetime(pmi_cx_df['日期']).dt.strftime("%Y-%m")
# x_data = list(range(1994,2023))
y_data = pmi_cx_df['制造业PMI'].values.tolist()
y_stock_data = stock_zh_index_monthly_df.loc[pmi_cx_df['日期'].iloc[0]:][:len(pmi_cx_df)]['close'].to_list()  
(
    Line(init_opts=opts.InitOpts(
        width='1200px', 
        height='600px', 
        theme='white',
        # bg_color='rgba(123, 200, 88, 0.4)'
        )
    )
    .add_xaxis(xaxis_data=x_data)

    .add_yaxis(
        series_name="制造业PMI",
        y_axis=y_data,
        label_opts=opts.LabelOpts(is_show=False),
        yaxis_index=0
    )

    .add_yaxis(
        series_name="上证指数",
        y_axis=y_stock_data,
        label_opts=opts.LabelOpts(is_show=False),
        yaxis_index=1
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            # title="我是主标题", 
            # subtitle='我是副标题',
            title_textstyle_opts=opts.TextStyleOpts(color='white'),
            subtitle_textstyle_opts=opts.TextStyleOpts(color='gray')
        ),
        legend_opts=opts.LegendOpts(is_show=True, textstyle_opts=opts.TextStyleOpts(color='white')),
        tooltip_opts=opts.TooltipOpts(
            is_show=True,
            trigger_on="mousemove|click",
            axis_pointer_type='cross'),
        xaxis_opts=opts.AxisOpts(
            type_="category",
            splitline_opts=opts.SplitLineOpts(is_show=False, linestyle_opts=opts.LineStyleOpts(type_='dotted'))
        ),
        yaxis_opts=opts.AxisOpts(
            type_="value",
            min_ = 20,
            max_ = 60,
            # axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=False, linestyle_opts=opts.LineStyleOpts(type_='dotted')),
        ),
        datazoom_opts=opts.DataZoomOpts(range_start=3000, range_end=100),
    )
    
    .set_series_opts(
        # 为了不影响标记点，这里把标签关掉
        label_opts=opts.LabelOpts(is_show=False),
        markline_opts=opts.MarkLineOpts(
            data=[
                opts.MarkLineItem(y=50, name="荣枯线"),
                # opts.MarkLineItem(type_="average", name="平均值"),
            ]
        ),
        # markpoint_opts=opts.MarkPointOpts(
        #     data=[
        #         opts.MarkPointItem(type_="max", name="最大值"),
        #         opts.MarkPointItem(type_="min", name="最小值"),
        #         opts.MarkPointItem(type_="average", name="平均值"),
        #     ])
    )


    .extend_axis(
        yaxis=opts.AxisOpts(
            type_="value",
            min_=2000,  # 最小值
            max_=5000,  # 最大值
            # is_show=False, 
            splitline_opts=opts.SplitLineOpts(is_show=False, linestyle_opts=opts.LineStyleOpts(type_='dotted'))
        )
    )
    .render_notebook()
)